In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [0]:
# read data from bronze layer
df_hosa=spark.read.parquet('/mnt/bronze/hosa/providers')
df_hosb=spark.read.parquet('/mnt/bronze/hosb/providers')

df_merged=df_hosa.unionByName(df_hosb)

df_merged.createOrReplaceGlobalTempView('providers')

In [0]:
%sql
use catalog emrcatalog;

In [0]:
%sql
create table if not exists silver.providers 
(
  ProviderID string,
  FirstName string,
  LastName string,
  Specialization string,
  DeptID string,
  NPI long,
  datasource string,
  is_quarantine boolean -- if some rows fail data quailty check, we quarantine them, keep them seperate
)
using delta;

In [0]:
%sql
truncate table silver.providers;

In [0]:
%sql
insert into silver.providers
select ProviderID,FirstName,LastName,Specialization,DeptID,cast(NPI as int),datasource,
case when ProviderID is NULL or DeptID is NULL then true else false end as is_quarantine
from global_temp.providers;

In [0]:
%sql
select * from silver.providers limit 10;